In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('G:\Github\Recipe_reco\data\RAW_recipes.csv')

In [3]:
data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [4]:
convert_to_list = ['tags','nutrition','steps','ingredients']

for c in convert_to_list:
    data[c] = data[c].apply(lambda x: eval(x))

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
name              231636 non-null object
id                231637 non-null int64
minutes           231637 non-null int64
contributor_id    231637 non-null int64
submitted         231637 non-null object
tags              231637 non-null object
nutrition         231637 non-null object
n_steps           231637 non-null int64
steps             231637 non-null object
description       226658 non-null object
ingredients       231637 non-null object
n_ingredients     231637 non-null int64
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [6]:
data.to_pickle('data/raw_df.pkl')

In [12]:
def treat_ingredients(ing_list):
    output = []
    for ingredient in ing_list:
        ingredient_list = ingredient.split(' ')
        output.append("_".join(ingredient_list))
    return output
ingredients_all = data.ingredients.apply(lambda x: treat_ingredients(x))

In [13]:
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
ingredients_all = data.ingredients.apply(lambda x: ", ".join(x))

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(token_pattern='(?u)\\b\\w\\w\\w+\\b')

# Fit and transform
X = vect.fit_transform(ingredients_all)

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())


In [14]:
#fit LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus,num_topics = 25,passes = 20, random_state = 0, id2word = id_map)

In [15]:
ldamodel.print_topics(num_topics = 25,num_words = 15)

[(0,
  '0.091*"sauce" + 0.065*"ground" + 0.062*"pepper" + 0.050*"onion" + 0.050*"beef" + 0.047*"powder" + 0.047*"garlic" + 0.044*"tomato" + 0.040*"salt" + 0.029*"chili" + 0.028*"worcestershire" + 0.021*"beans" + 0.020*"water" + 0.020*"hot" + 0.017*"lean"'),
 (1,
  '0.065*"sauce" + 0.059*"oil" + 0.043*"garlic" + 0.042*"soy" + 0.030*"fresh" + 0.029*"red" + 0.026*"ginger" + 0.025*"pepper" + 0.025*"sesame" + 0.024*"rice" + 0.021*"green" + 0.021*"powder" + 0.020*"sugar" + 0.020*"chili" + 0.018*"cloves"'),
 (2,
  '0.104*"flour" + 0.097*"baking" + 0.084*"sugar" + 0.063*"salt" + 0.060*"powder" + 0.051*"all" + 0.051*"purpose" + 0.046*"butter" + 0.043*"soda" + 0.040*"eggs" + 0.039*"vanilla" + 0.027*"milk" + 0.024*"egg" + 0.023*"extract" + 0.018*"oil"'),
 (3,
  '0.199*"ground" + 0.134*"cinnamon" + 0.079*"nutmeg" + 0.044*"ginger" + 0.036*"cloves" + 0.036*"pumpkin" + 0.033*"allspice" + 0.029*"raisins" + 0.023*"oil" + 0.021*"apple" + 0.020*"cake" + 0.020*"spice" + 0.018*"cardamom" + 0.016*"water" + 